<a href="https://colab.research.google.com/github/DavidAntonio7/criacao_df_brasileirao/blob/main/criando_DataFrame_brasileirao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import seaborn as sns
from datetime import datetime

In [67]:
df = pd.read_csv('df_base.csv',sep = ';')
df.head()

,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor
0,1,29/03/2003,16h00,Sábado,Athlético-PR,Grêmio,Athlético-PR,Arena da Baixada,2,0,PR,RS,PR
1,1,29/03/2003,16h00,Sábado,Guarani,Vasco,Guarani,Brinco de Ouro,4,2,SP,RJ,SP
2,1,30/03/2003,16h00,Domingo,Fortaleza,Bahia,-,Castelão,0,0,CE,BA,-
3,1,30/03/2003,16h00,Domingo,Flamengo,Coritiba,-,Maracanã,1,1,RJ,PR,-
4,1,30/03/2003,16h00,Domingo,Vitória,Figueirense,-,Barradão,1,1,ES,SC,-


In [68]:
df.drop(columns=['Horário','Dia','Arena','Estado Mandante','Estado Visitante', 'Estado Vencedor'],inplace=True)

In [69]:
df1 = df.copy()

In [70]:
##tratando as datas para que apareca somente o ano da temporada
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y').dt.year

dados faltantes

In [71]:
colunas = ['Rodada', 'Data', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar']
jogos = [
    [2,2008,'Cruzeiro','Botafogo-RJ','Cruzeiro',1,0],
    [2,2008,'Sport','Vitória','-',0,0],
    [2,2008,'Vasco','Portuguesa','-',3,1]
        ]
faltantes_2008_2 = pd.DataFrame(jogos,columns=colunas)
df = pd.concat([df,faltantes_2008_2])

In [72]:
df_mandante = df.copy()
df_visitante = df.copy()

In [73]:
df_mandante['Time'] = df_mandante.Mandante
df_mandante['Mando_de_campo'] = 1
df_visitante['Time'] = df_visitante.Visitante
df_visitante['Mando_de_campo'] = -1

In [74]:
df_visitante.index = range(df_visitante.shape[0],df_visitante.shape[0]+df_visitante.shape[0])
df_times = pd.concat([df_visitante,df_mandante])
df_times.shape

(14538, 9)

In [75]:
df_times1 = df_times.copy()

In [76]:
df_times = df_times1.copy()
df_times.rename(columns={'Data' : 'Temporada'},inplace=True)
df_times.head(1)

,Rodada,Temporada,Mandante,Visitante,Vencedor,Mandante Placar,Visitante Placar,Time,Mando_de_campo
7269,1,2003,Athlético-PR,Grêmio,Athlético-PR,2,0,Grêmio,-1


In [77]:
df_times.columns

Index(['Rodada', 'Temporada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Time', 'Mando_de_campo'],
      dtype='object')

In [78]:
df_times = df_times[['Time','Temporada','Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar','Mando_de_campo']]

##funcao para calcular a tabela a partir dos resultados

In [79]:
def calculaTabela(x):    
    pontos = 0;vitorias = 0;derrotas = 0;empates = 0;gols_pro = 0;gols_contra = 0;vitorias_consecutivas = 0;
    derrotas_consecutivas = 0;jogos_marcando = 0;jogos_sem_sofrer_gols = 0;vencedor = 0
    for i in range(0,x.shape[0]):
        ##se o time for visitante
        if x.iloc[i].Time == x.iloc[i].Visitante:
            ##pontos - vitorias - derrotas - empates - vitorias concecutivas - derrotas concecutivas
            if x.iloc[i].Visitante == x.iloc[i].Vencedor: #vencedor
                vitorias += 1;vitorias_consecutivas += 1; derrotas_consecutivas = 0; pontos+=3
            elif x.iloc[i].Mandante == x.iloc[i].Vencedor:
                derrotas+= 1;vitorias_consecutivas = 0; derrotas_consecutivas +=1; pontos+=0
            else:
                empates+= 1;vitorias_consecutivas = 0; derrotas_consecutivas = 0; pontos+=1
            ##metricas de gols 
            gols_pro += x.iloc[i]['Visitante Placar']; gols_contra += x.iloc[i]['Mandante Placar'];
            if x.iloc[i]['Visitante Placar'] > 0:
                if jogos_marcando >= 0:
                    jogos_marcando+=1
                else:
                    jogos_marcando = 1
            elif x.iloc[i]['Visitante Placar'] <= 0:
                if jogos_marcando >= 0:
                    jogos_marcando = -1
                else:
                    jogos_marcando += -1
            if x.iloc[i]['Mandante Placar'] > 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = -1
                else:
                    jogos_sem_sofrer_gols -= 1
            elif x.iloc[i]['Mandante Placar'] == 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = +1
                else:
                    jogos_sem_sofrer_gols += 1
        ##se o time for mandante
        if x.iloc[i].Time == x.iloc[i].Mandante:
            ##pontos - vitorias - derrotas - empates - vitorias concecutivas - derrotas concecutivas
            if x.iloc[i].Mandante == x.iloc[i].Vencedor: #vencedor
                vitorias += 1;vitorias_consecutivas += 1; derrotas_consecutivas = 0; pontos+=3
            elif x.iloc[i].Visitante == x.iloc[i].Vencedor:
                derrotas+= 1;vitorias_consecutivas = 0; derrotas_consecutivas +=1; pontos+=0
            else:
                empates+= 1;vitorias_consecutivas = 0; derrotas_consecutivas = 0; pontos+=1
            ##metricas de gols 
            gols_pro += x.iloc[i]['Mandante Placar']; gols_contra += x.iloc[i]['Visitante Placar'];
            if x.iloc[i]['Mandante Placar'] > 0:
                if jogos_marcando >= 0:
                    jogos_marcando+=1
                else:
                    jogos_marcando = 1
            elif x.iloc[i]['Mandante Placar'] <= 0:
                if jogos_marcando >= 0:
                    jogos_marcando = -1
                else:
                    jogos_marcando += -1
            if x.iloc[i]['Visitante Placar'] > 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = -1
                else:
                    jogos_sem_sofrer_gols -= 1
            elif x.iloc[i]['Visitante Placar'] == 0:
                if jogos_sem_sofrer_gols >= 0:
                    jogos_sem_sofrer_gols = +1
                else:
                    jogos_sem_sofrer_gols = 1
        x.iloc[i,9:] = [pontos,vitorias,derrotas,empates,gols_pro,gols_contra,(gols_pro-gols_contra),vitorias_consecutivas,derrotas_consecutivas,
    jogos_marcando,jogos_sem_sofrer_gols]
    return x
                

In [80]:
#ordenando as colunas
df_times = df_times[['Time','Temporada','Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar','Mando_de_campo']]

In [81]:
#criando novas colunas
df_times[['Pontos','Vitoria','Derrota','Empate','Gols_pro','Gols_contra','Saldo_de_gols',
   'Vitorias_consecutivas','Derrotas_consecutivas','Jogos_marcando','Jogos_sem_sofrer_gols']] = 0

In [82]:
#ordenando a base
df_times.sort_values(['Temporada','Rodada'],inplace=True)

In [83]:
#removendo as temporadas com mais de 20 times e outras com erros 
df_times['Temporada'] = df_times['Temporada'].astype(int)
df_times = df_times[df_times['Temporada'] > 2007]

In [84]:
df_times2 = df_times.copy()

##consertando valores escritos em maiuscula

In [85]:
df_times.Time = df_times.Time.str.lower()
df_times.Mandante = df_times.Mandante.str.lower()
df_times.Visitante = df_times.Visitante.str.lower()
df_times.Vencedor = df_times.Vencedor.str.lower()


<ipython-input-85-b68301d8db7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_times.Time = df_times.Time.str.lower()
<ipython-input-85-b68301d8db7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_times.Mandante = df_times.Mandante.str.lower()
<ipython-input-85-b68301d8db7b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [86]:
#conferindo o numero de clubes por temporada
for temp in range(2008,2021):
    print(temp ,df_times[df_times.Temporada == temp].Time.nunique(),df_times[df_times.Temporada == temp].Mandante.nunique(),
          df_times[df_times.Temporada == temp].Visitante.nunique())

2008 20 20 20
2009 20 20 20
2010 20 20 20
2011 20 20 20
2012 20 20 20
2013 20 20 20
2014 20 20 20
2015 20 20 20
2016 20 20 20
2017 20 20 20
2018 20 20 20
2019 20 20 20
2020 20 20 20


Calculando a tabela para todos times em todas temporadas

In [87]:
for temporada in range(2008, 2022):
    for time in df_times[df_times['Temporada'] == temporada]['Time'].unique():
        selecao = df_times[(df_times['Temporada'] == temporada) & (df_times['Time'] == time)].copy()
        resultado = calculaTabela(selecao)
        df_times.loc[(df_times['Temporada'] == temporada) & (df_times['Time'] == time)] = resultado

as temporadas de 2021 e 2020 estao faltando dados -> romovendo-as do df

In [88]:
df_times = df_times.query('Temporada < 2020')

## tratando a posicao na tabela

In [89]:
#ordenando o df segundo os criterios do brasileirao
df_times.sort_values(['Temporada','Rodada','Pontos','Vitoria','Saldo_de_gols'],ascending=[True,True,False,False,False],inplace=True)

<ipython-input-89-e4f4b8afba5c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_times.sort_values(['Temporada','Rodada','Pontos','Vitoria','Saldo_de_gols'],ascending=[True,True,False,False,False],inplace=True)


In [90]:
df_times.query('Temporada == 2010').tail(20)

,Time,Temporada,Rodada,Mandante,Visitante,Vencedor,Mandante Placar,Visitante Placar,Mando_de_campo,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
3462,fluminense,2010,38,fluminense,guarani,fluminense,1,0,1,71,20,7,11,62,36,26,3,0,5,1
3461,cruzeiro,2010,38,cruzeiro,palmeiras,cruzeiro,2,1,1,69,20,9,9,53,38,15,3,0,3,-5
10727,corinthians,2010,38,goiás,corinthians,-,1,1,-1,68,19,8,11,65,41,24,0,0,8,-1
3463,grêmio,2010,38,grêmio,botafogo-rj,grêmio,3,0,1,63,17,9,12,68,43,25,3,0,3,1
3460,athlético-pr,2010,38,athlético-pr,avaí,athlético-pr,1,0,1,60,17,12,9,43,45,-2,1,0,9,1
10732,botafogo-rj,2010,38,grêmio,botafogo-rj,grêmio,3,0,-1,59,14,7,17,54,42,12,0,1,-1,-4
10725,internacional,2010,38,grêmio prudente,internacional,internacional,0,3,-1,58,16,12,10,48,41,7,1,0,5,-3
3459,santos,2010,38,santos,flamengo,-,0,0,1,56,15,12,11,63,50,13,0,0,-1,1
3464,são paulo,2010,38,são paulo,atlético-mg,são paulo,4,0,1,55,15,13,10,54,54,0,1,0,4,1
10730,palmeiras,2010,38,cruzeiro,palmeiras,cruzeiro,2,1,-1,50,12,12,14,42,43,-1,0,4,2,-4


In [91]:
df_times.columns

Index(['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo', 'Pontos',
       'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols'],
      dtype='object')

In [92]:
df_times['Posicao'] = 0

<ipython-input-92-db506841635c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_times['Posicao'] = 0


In [93]:
#organizando
df_times = df_times[['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo',  'Posicao','Pontos',
       'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols']]

In [94]:
df_times1 = df_times.copy()

In [95]:
# conferindo se o numeros estao correto 
df_times.groupby('Temporada').Time.count()

Temporada
2008    760
2009    760
2010    760
2011    760
2012    760
2013    760
2014    760
2015    760
2016    760
2017    760
2018    760
2019    760
Name: Time, dtype: int64

In [96]:
for temp,rodada in zip (df_times.Temporada.unique(), range(1,39)):
    df_times.loc[(df_times.Temporada == temp) & (df_times.Rodada == 38), 'Posicao'] = range(1, 21)

In [97]:
df_times[(df_times.Temporada == 2011) & (df_times.Rodada == 38)].Posicao

3839      1
3837      2
11107     3
11106     4
3844      5
3845      6
11105     7
11110     8
3838      9
11114    10
11108    11
11113    12
3840     13
3842     14
11112    15
3843     16
3841     17
11111    18
11109    19
3836     20
Name: Posicao, dtype: int64

In [98]:
#arrumando os valores dos index
df_times.index = range(0,df_times.shape[0])

fazendo uma coluna adversario

In [99]:
Adversario = []
for i in df_times.index:
    if df_times.iloc[i].Time == df_times.iloc[i].Mandante:
        Adversario.append(df_times.iloc[i].Visitante)
    else:
        Adversario.append(df_times.iloc[i].Mandante)

In [100]:
df_times['Adversario'] = Adversario

In [101]:
df_times[['Time','Adversario']].head()

,Time,Adversario
0,cruzeiro,vitória
1,botafogo-rj,sport
2,coritiba,palmeiras
3,flamengo,santos
4,grêmio,são paulo


In [102]:
df_times.columns

Index(['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo', 'Posicao',
       'Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols', 'Adversario'],
      dtype='object')

#.

In [103]:
df_times.head()

,Time,Temporada,Rodada,Mandante,Visitante,Vencedor,Mandante Placar,Visitante Placar,Mando_de_campo,Posicao,...,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols,Adversario
0,cruzeiro,2008,1,vitória,cruzeiro,cruzeiro,0,2,-1,0,...,0,0,2,0,2,1,0,1,1,vitória
1,botafogo-rj,2008,1,botafogo-rj,sport,botafogo-rj,2,0,1,0,...,0,0,2,0,2,1,0,1,1,sport
2,coritiba,2008,1,coritiba,palmeiras,coritiba,2,0,1,0,...,0,0,2,0,2,1,0,1,1,palmeiras
3,flamengo,2008,1,flamengo,santos,flamengo,3,1,1,0,...,0,0,3,1,2,1,0,1,-1,santos
4,grêmio,2008,1,são paulo,grêmio,grêmio,0,1,-1,0,...,0,0,1,0,1,1,0,1,1,são paulo


In [104]:
colunas_ad = ['Posicao','Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols']
colunas_adversario = ['Adversario_'+i for i in colunas_ad]


In [105]:
df_times = pd.merge(df_times,df_times,left_on=['Temporada','Rodada','Adversario'],right_on=['Temporada','Rodada','Time'],suffixes=['','_Visitante'])

In [106]:
df_times.head()

,Time,Temporada,Rodada,Mandante,Visitante,Vencedor,Mandante Placar,Visitante Placar,Mando_de_campo,Posicao,...,Derrota_Visitante,Empate_Visitante,Gols_pro_Visitante,Gols_contra_Visitante,Saldo_de_gols_Visitante,Vitorias_consecutivas_Visitante,Derrotas_consecutivas_Visitante,Jogos_marcando_Visitante,Jogos_sem_sofrer_gols_Visitante,Adversario_Visitante
0,cruzeiro,2008,1,vitória,cruzeiro,cruzeiro,0,2,-1,0,...,1,0,0,2,-2,0,1,-1,-1,cruzeiro
1,botafogo-rj,2008,1,botafogo-rj,sport,botafogo-rj,2,0,1,0,...,1,0,0,2,-2,0,1,-1,-1,botafogo-rj
2,coritiba,2008,1,coritiba,palmeiras,coritiba,2,0,1,0,...,1,0,0,2,-2,0,1,-1,-1,coritiba
3,flamengo,2008,1,flamengo,santos,flamengo,3,1,1,0,...,1,0,1,3,-2,0,1,1,-1,flamengo
4,grêmio,2008,1,são paulo,grêmio,grêmio,0,1,-1,0,...,1,0,0,1,-1,0,1,-1,-1,grêmio


##criando a tabela Rodadas

In [107]:
df_times.shape

(9120, 42)

In [108]:
df_rodadas = df_times[['Temporada', 'Rodada', 'Mandante', 'Visitante','Mandante Placar', 'Visitante Placar']].copy()

In [109]:
#os mandantes sejam os primeiros valores
df_rodadas.sort_values(['Temporada', 'Rodada', 'Mandante'],inplace = True)

In [110]:
df_rodadas.drop_duplicates(['Temporada','Rodada','Mandante'], inplace = True)

In [111]:
df_rodadas.shape

(4560, 6)

In [112]:
df_rodadas.to_csv('rodadas_entre_2008_2019',index = False)

##criando as tabelas mandantes e visitantes

In [113]:
df_times.columns

Index(['Time', 'Temporada', 'Rodada', 'Mandante', 'Visitante', 'Vencedor',
       'Mandante Placar', 'Visitante Placar', 'Mando_de_campo', 'Posicao',
       'Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols', 'Adversario',
       'Time_Visitante', 'Mandante_Visitante', 'Visitante_Visitante',
       'Vencedor_Visitante', 'Mandante Placar_Visitante',
       'Visitante Placar_Visitante', 'Mando_de_campo_Visitante',
       'Posicao_Visitante', 'Pontos_Visitante', 'Vitoria_Visitante',
       'Derrota_Visitante', 'Empate_Visitante', 'Gols_pro_Visitante',
       'Gols_contra_Visitante', 'Saldo_de_gols_Visitante',
       'Vitorias_consecutivas_Visitante', 'Derrotas_consecutivas_Visitante',
       'Jogos_marcando_Visitante', 'Jogos_sem_sofrer_gols_Visitante',
       'Adversario_Visitante'],
      dtype='object')

In [114]:
dados_Mandante = df_times.sort_values(['Temporada', 'Rodada', 'Mandante'])[
    ['Time','Temporada', 'Rodada','Posicao',
       'Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols'
       ]
]

In [115]:
dados_visitante = df_times.sort_values(['Temporada', 'Rodada', 'Mandante'])[
    [
       'Visitante','Temporada', 'Rodada','Posicao_Visitante', 'Pontos_Visitante', 'Vitoria_Visitante',
       'Derrota_Visitante', 'Empate_Visitante', 'Gols_pro_Visitante',
       'Gols_contra_Visitante', 'Saldo_de_gols_Visitante',
       'Vitorias_consecutivas_Visitante', 'Derrotas_consecutivas_Visitante',
       'Jogos_marcando_Visitante', 'Jogos_sem_sofrer_gols_Visitante',
       ]
]

In [116]:
dados_Mandante = dados_Mandante.loc[df_rodadas.index]
dados_visitante = dados_visitante.loc[df_rodadas.index]

In [117]:
dados_visitante.set_axis( ['Time','Temporada', 'Rodada','Posicao',
       'Pontos', 'Vitoria', 'Derrota', 'Empate', 'Gols_pro', 'Gols_contra',
       'Saldo_de_gols', 'Vitorias_consecutivas', 'Derrotas_consecutivas',
       'Jogos_marcando', 'Jogos_sem_sofrer_gols'], axis=1,inplace=True
                       )

<ipython-input-117-28e113b25ea4>:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  dados_visitante.set_axis( ['Time','Temporada', 'Rodada','Posicao',


Conferindo

In [118]:
df_rodadas.head()

,Temporada,Rodada,Mandante,Visitante,Mandante Placar,Visitante Placar
9,2008,1,atlético-mg,fluminense,0,0
1,2008,1,botafogo-rj,sport,2,0
2,2008,1,coritiba,palmeiras,2,0
3,2008,1,flamengo,santos,3,1
7,2008,1,internacional,vasco,1,0


In [119]:
dados_Mandante.head()

,Time,Temporada,Rodada,Posicao,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
9,fluminense,2008,1,0,1,0,0,1,0,0,0,0,0,-1,1
1,botafogo-rj,2008,1,0,3,1,0,0,2,0,2,1,0,1,1
2,coritiba,2008,1,0,3,1,0,0,2,0,2,1,0,1,1
3,flamengo,2008,1,0,3,1,0,0,3,1,2,1,0,1,-1
7,internacional,2008,1,0,3,1,0,0,1,0,1,1,0,1,1


In [120]:
dados_visitante.head()

,Time,Temporada,Rodada,Posicao,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
9,fluminense,2008,1,0,1,0,0,1,0,0,0,0,0,-1,1
1,sport,2008,1,0,0,0,1,0,0,2,-2,0,1,-1,-1
2,palmeiras,2008,1,0,0,0,1,0,0,2,-2,0,1,-1,-1
3,santos,2008,1,0,0,0,1,0,1,3,-2,0,1,1,-1
7,vasco,2008,1,0,0,0,1,0,0,1,-1,0,1,-1,-1


In [121]:
df_times_final = pd.concat([dados_Mandante,dados_visitante])

In [124]:
df_times_final.head()

,Time,Temporada,Rodada,Posicao,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
9,fluminense,2008,1,0,1,0,0,1,0,0,0,0,0,-1,1
1,botafogo-rj,2008,1,0,3,1,0,0,2,0,2,1,0,1,1
2,coritiba,2008,1,0,3,1,0,0,2,0,2,1,0,1,1
3,flamengo,2008,1,0,3,1,0,0,3,1,2,1,0,1,-1
7,internacional,2008,1,0,3,1,0,0,1,0,1,1,0,1,1


In [129]:
df_times_final.sort_values(['Temporada','Rodada','Posicao'],inplace=True)

In [130]:
df_times_final.tail(20)

,Time,Temporada,Rodada,Posicao,Pontos,Vitoria,Derrota,Empate,Gols_pro,Gols_contra,Saldo_de_gols,Vitorias_consecutivas,Derrotas_consecutivas,Jogos_marcando,Jogos_sem_sofrer_gols
9100,flamengo,2019,38,1,90,28,4,6,86,37,49,0,1,-1,-6
9100,flamengo,2019,38,2,74,22,8,8,60,33,27,1,0,1,1
9102,palmeiras,2019,38,3,74,21,6,11,61,32,29,2,0,3,-6
9103,grêmio,2019,38,4,65,19,11,8,64,39,25,0,1,3,-1
9104,athlético-pr,2019,38,5,64,18,10,10,51,32,19,0,0,-1,1
9105,são paulo,2019,38,6,63,17,9,12,39,30,9,2,0,2,-3
9106,internacional,2019,38,7,57,16,13,9,44,39,5,1,0,5,-8
9107,corinthians,2019,38,8,56,14,10,14,42,34,8,0,1,4,-1
9108,fortaleza,2019,38,9,53,15,15,8,50,49,1,1,0,1,-3
9103,grêmio,2019,38,10,52,15,16,7,46,64,-18,1,0,6,-8


exportando 

In [131]:
df_rodadas.to_csv('rodadas_brasileirao_2008_2019.csv', index = False)
df_times_final.to_csv('dados_brasileirao_2008_2019.csv', index = False)


